In [1]:
import os
os.getcwd()
os.chdir("E:\Afitech PPML Datasets")
os.getcwd()

'E:\\Afitech PPML Datasets'

In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [3]:
stopwords=nltk.corpus.stopwords.words('english')
ps=nltk.PorterStemmer()

import pandas as pd
data=pd.read_csv("SMSSpamCollection.tsv",sep='\t')
data.columns=['label','body_text']
data.head()

,label,body_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,ham,"Nah I don't think he goes to usf, he lives aro..."
2,ham,Even my brother is not like to speak with me. ...
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
4,ham,As per your request 'Melle Melle (Oru Minnamin...


In [4]:
def count_punct(text):
    count=sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")),3)*100


data['body_len']=data['body_text'].apply(lambda x:len(x)-x.count(" "))
data['puc%']=data['body_text'].apply(lambda x:count_punct(x))


def clean_text(text):
    text="".join([word.lower() for word in text if word not in string.punctuation])
    tokens=re.split('\W+',text)
    text=[ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [5]:
tfidf_vect=TfidfVectorizer(analyzer=clean_text)
x_fd=tfidf_vect.fit_transform(data['body_text'])
x_feature=pd.concat([data['body_len'],data['puc%'],pd.DataFrame(x_fd.toarray())],axis=1)
x_feature.head()

,body_len,puc%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn.ensemble import RandomForestClassifier
print(RandomForestClassifier())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators='warn',
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [16]:
from sklearn.model_selection import KFold,cross_val_score
rf=RandomForestClassifier(n_jobs=-1)
k_fold=KFold(n_splits=5)
cross_val_score(rf,x_feature,data['label'],cv=k_fold,scoring='accuracy',n_jobs=-1)


array([0.96588869, 0.97217235, 0.96765499, 0.9541779 , 0.97124888])

In [8]:
x=x_feature
y=data['label']

In [17]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [18]:
rf=RandomForestClassifier(n_estimators=20,max_depth=100,n_jobs=-1)
rf_model=rf.fit(X_train,y_train)

In [19]:
rf_model.feature_importances_[]

array([0.03836412, 0.0101995 , 0.0006165 , ..., 0.00017416, 0.        ,
       0.        ])

In [23]:
sorted(zip(rf_model.feature_importances_,X_train.columns),reverse=True)[0:10]

[(0.03836412005278937, 'body_len'),
 (0.03495136440666929, 3134),
 (0.0319766566146737, 1803),
 (0.026997016522212425, 6285),
 (0.022838816001627134, 2031),
 (0.019104594273683047, 5988),
 (0.019078666318100235, 7350),
 (0.01682889756906266, 4796),
 (0.014792973105259085, 6746),
 (0.014755595772554402, 1361)]

In [39]:
y_pred=rf_model.predict(X_test)
precision,recall,fscore,support=score(y_test,y_pred,pos_label='spam',average='binary')


In [40]:
print('Precision: {} /Recall: {} /accuracy: {}'.format(round(precision,3),
                                                       round(recall,3),
                                                       round((y_pred==y_test).sum()/len(y_pred),3)))

Precision: 0.992 /Recall: 0.85 /accuracy: 0.979


In [24]:
#grid searchtree


In [46]:
def train_RF(n_est,depth):
    rf=RandomForestClassifier(n_estimators=n_est,max_depth=depth,n_jobs=-1)
    rf_model=rf.fit(X_train,y_train)
    y_pred=rf_model.predict(X_test)
    precision,recall,fscore,support=score(y_test,y_pred,pos_label='spam',average='binary')
    print('Est: {}/depth:{}.......Precision: {} /accuarcy :{}'.format(n_est,depth,\
                                                                      round(precision,3),round(recall,3),round(y_pred==y_test).sum()/len(y_pred),3))

In [47]:
for n_est in [10,50,100]:
    for depth in [10,20,30, None]:
        train_RF(n_est,depth)

Est: 10/depth:10.......Precision: 1.0 /accuarcy :0.143
Est: 10/depth:20.......Precision: 1.0 /accuarcy :0.565
Est: 10/depth:30.......Precision: 1.0 /accuarcy :0.714
Est: 10/depth:None.......Precision: 0.983 /accuarcy :0.782
Est: 50/depth:10.......Precision: 1.0 /accuarcy :0.32
Est: 50/depth:20.......Precision: 1.0 /accuarcy :0.626
Est: 50/depth:30.......Precision: 1.0 /accuarcy :0.721
Est: 50/depth:None.......Precision: 1.0 /accuarcy :0.837
Est: 100/depth:10.......Precision: 1.0 /accuarcy :0.279
Est: 100/depth:20.......Precision: 1.0 /accuarcy :0.612
Est: 100/depth:30.......Precision: 1.0 /accuarcy :0.735
Est: 100/depth:None.......Precision: 1.0 /accuarcy :0.837
